In [1]:
# importing pandas and numpy
import pandas as pd
import numpy as np
import collections
import itertools as it
import time
import re
support=50
confidence = 0.7

In [2]:

data = pd.read_csv('association-rule-test-data.txt', sep="\t", header=None)
data.head(2)

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,Up,Up,Down,Up,Down,Up,Up,Down,Down,Up,...,Up,Up,Down,Up,Down,Down,Down,Up,Down,ALL
1,Up,Down,Up,Down,Up,Down,Down,Down,Down,Up,...,Up,Up,Down,Down,Down,Down,Down,Up,Up,ALL


In [3]:
# Appending G1 to the data
for i in range(len(data.columns)-1):
  data[:][i] = "G"+str(i+1)+ "_"+data.iloc[:][i]
data.head(2)

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,G1_Up,G2_Up,G3_Down,G4_Up,G5_Down,G6_Up,G7_Up,G8_Down,G9_Down,G10_Up,...,G92_Up,G93_Up,G94_Down,G95_Up,G96_Down,G97_Down,G98_Down,G99_Up,G100_Down,ALL
1,G1_Up,G2_Down,G3_Up,G4_Down,G5_Up,G6_Down,G7_Down,G8_Down,G9_Down,G10_Up,...,G92_Up,G93_Up,G94_Down,G95_Down,G96_Down,G97_Down,G98_Down,G99_Up,G100_Up,ALL


In [4]:
# column_values = data.iloc[:,:-1].values.ravel()
# unique_values =  pd.unique(column_values)

In [5]:
freq_item_set = {}
last_freq_item_sets = {}

In [6]:
c={}
for i in range(len(data.columns)):
  c.update(collections.Counter(data.iloc[:,i].values))

In [7]:
start_time = time.time()
from collections import defaultdict
freq_item=[]
dict2={}
for key in c:
  if c[key] >= support:
    freq_item.append(tuple([key]))
    dict2[tuple([key])] = c[key]
print(len(freq_item))


# for key in c.keys():
#   k = tuple([key])
#   dict2[k]=c[key]
dict1=defaultdict(list)
for key in dict2.keys():
  for i in range(len(data)):
    if key in data.loc[i,:].values:
      dict1[key].append(i)
temp = list(dict1.keys())
for key in temp:
  if key not in dict2.keys():
    del dict1[key]
print("--- %s seconds ---" % (time.time() - start_time))
freq_item_set.update(dict2)

109
--- 1.360398530960083 seconds ---


In [8]:

def generate_combinations(f,length):
  start_time = time.time()
  comb_freq_items = []
  for i in it.combinations(f,2):
    temp=tuple(sorted(set(i[0]+i[1])))
    # print(i[0]+i[1])
    if len(temp) == length:
      comb_freq_items.append(temp)
  comb_freq_items=set(comb_freq_items)
  print("---generate_combinations %s seconds ---" % (time.time() - start_time))
  return comb_freq_items

In [9]:

def func_dict(dict1,dict_count,items_freq):
  start_time = time.time()  
  d1=defaultdict(list)  # gives rows of each n-1 len combo
  d2={}     # gives count of each n-1 len combo
  # print(items_freq)
  for i in items_freq:
    count = 0
    # print(i)
    if i[:-1] in dict1.keys():
      col_num1 = re.findall("\d+", i[-1])
      if col_num1 ==[]:
        col_num = len(data.columns)
      else:
        col_num = int(col_num1[0])
      for j in dict1[i[:-1]]:
        if i[-1] == data[col_num-1][j]:
          d1[i].append(j)
          count = count + 1
          # print(count)
      if count >= support:
        # print(count)
        d2[i] = count
  freq_item_set.update(d2)
  temp = list(d1.keys())
  for key in temp:
    if key not in d2.keys():
      del d1[key]
  print("---func_dict %s seconds ---" % (time.time() - start_time))
  
  return (d1,d2)


In [10]:
%%time
i=1
support_count_dict={}

while(freq_item):
  i=i+1
  comb_freq_items=generate_combinations(freq_item,i)
  # print(comb_freq_items)
  freq_item=[]
  (dict1,dict2) = func_dict(dict1,dict2,comb_freq_items)
  # print(dict1)
  # print(dict2)
      # support_count_dict[j]=count_comb
  freq_item = list(dict2.keys())
  print(len(freq_item))
  if freq_item != []:
    last_freq_item_sets = freq_item



---generate_combinations 0.005985260009765625 seconds ---
---func_dict 4.327472925186157 seconds ---
63
---generate_combinations 0.0009982585906982422 seconds ---
---func_dict 0.18944263458251953 seconds ---
2
---generate_combinations 0.0 seconds ---
---func_dict 0.0009975433349609375 seconds ---
0
Wall time: 4.53 s


In [11]:
len(freq_item_set)

174

In [12]:
last_freq_item_sets

[('G59_Up', 'G72_Up', 'G96_Down'), ('G59_Up', 'G72_Up', 'G82_Down')]

In [13]:
def subs(l):
    if l == []:
        return [[]]

    x = subs(l[1:])

    return x + [[l[0]] + y for y in x]

In [14]:
def func_conf(head,body):
  if (freq_item_set[tuple(sorted(head+body))]/freq_item_set[tuple(sorted(head))]) >= confidence:
    return (head,body)
  else:
    return ([],[])

In [15]:
head_list = []
body_list = []
rule_list = []
for k in last_freq_item_sets:
  rule_items=list(k)
  for li1 in subs(rule_items):
    if li1 == [] or li1 == rule_items:
      continue
    else:
      # print(set(i)-set(last_freq_item_sets[0]))
      body = [i for i in li1 + rule_items if i not in li1 or i not in rule_items]
      head = li1
      (head,body) = func_conf(head,body)
      if head != []:
        head_list.append(head)
        body_list.append(body)
        rule_list.append(head + ["->"] + body)

In [16]:
class asso_rule:
  def template1(entity,cardinality,items):
    HEAD = []
    count = 0
    if entity == "HEAD": 
      if cardinality == "ANY":
        for i in head_list:
          if any(item in items for item in i) == True:
            count = count+1
            HEAD.append(i)
        return HEAD,count
      elif cardinality == "NONE":
        for i in head_list:
          if any(item in items for item in i) == False:
            count = count+1
            HEAD.append(i)
        return HEAD,count
      else:
        for n in head_list:
          count = 0 
          for m in items: 
            if m in n: 
              count = count + 1
              if count == cardinality:
                HEAD.append(n)
                break  
        return HEAD,len(HEAD)
    elif entity == "BODY":
      if cardinality == "ANY":
        for i in body_list:
          if any(item in items for item in i) == True:
            count = count+1
            HEAD.append(i)
        return HEAD,count
      elif cardinality == "NONE":
        for i in body_list:
          if any(item in items for item in i) == False:
            count = count+1
            HEAD.append(i)
        return HEAD,count
      else:
        for n in body_list:
          count = 0 
          for m in items: 
            if m in n: 
              count = count + 1
              if count == cardinality:
                HEAD.append(n)
                break  
        return (HEAD,len(HEAD)) 
    elif entity == "RULE":
      if cardinality == "ANY":
        for i in rule_list:
          if any(item in items for item in i) == True:
            count = count+1
            HEAD.append(i)
        return HEAD,count
      elif cardinality == "NONE":
        for i in rule_list:
          if any(item in items for item in i) == False:
            count = count+1
            HEAD.append(i)
        return HEAD,count
      else:
        for n in rule_list:
          count = 0 
          for m in items: 
            if m in n: 
              count = count + 1
              if count == cardinality:
                HEAD.append(n)
                break  
        return (HEAD,len(HEAD))

  def template2(entity,cardinality):
    HEAD=[]
    if entity=="HEAD":
      for i in head_list:
        if len(i)>= cardinality:
          HEAD.append(i)
    elif entity=="BODY":
      for i in body_list:
        if len(i)>= cardinality:
          HEAD.append(i)
    elif entity=="RULE":
      for i in rule_list:
        if len(i)>= cardinality+1:
          HEAD.append(i)
    return HEAD,len(HEAD)
  
  # def template3("1or1", "HEAD", "ANY", ['G10_Down'], "BODY", 1, ['G59_UP'])
  def template3(clause,entity1, *l):
    con = re.findall("\D+", clause)[0]
    if len(l)== 5:
      cardinality1 = l[0] 
      item1 = l[1] 
      entity2 = l[2] 
      cardinality2 = l[3] 
      item2 = l[4]
      clause1_list,cnt1 = asso_rule.template1(entity1,cardinality1,item1)
      clause2_list,cnt2 = asso_rule.template1(entity2,cardinality2,item2) 
      clause1_tuple = tuple(tuple(sub) for sub in clause1_list)
      clause2_tuple = tuple(tuple(sub) for sub in clause2_list)
      if con == 'or':
        final_tuple = set(clause1_tuple) | set(clause2_tuple)
      elif con == 'and':
        final_tuple = set(clause1_tuple) & set(clause2_tuple)
    
    
    elif len(l)==4:
      cardinality1 = l[0] 
      item1 = l[1] 
      entity2 = l[2] 
      cardinality2 = l[3]
      clause1_list,cnt1 = asso_rule.template1(entity1,cardinality1,item1)
      clause2_list,cnt2 = asso_rule.template2(entity2,cardinality2) 
      clause1_tuple = tuple(tuple(sub) for sub in clause1_list)
      clause2_tuple = tuple(tuple(sub) for sub in clause2_list)
      if con == 'or':
        final_tuple = set(clause1_tuple) | set(clause2_tuple)
      elif con == 'and':
        final_tuple = set(clause1_tuple) & set(clause2_tuple)
      

    elif len(l)==3:
      cardinality1 = l[0] 
      entity2 = l[1] 
      cardinality2 = l[2]
      clause1_list,cnt1 = asso_rule.template2(entity1,cardinality1)
      clause2_list,cnt2 = asso_rule.template2(entity2,cardinality2) 
      clause1_tuple = tuple(tuple(sub) for sub in clause1_list)
      clause2_tuple = tuple(tuple(sub) for sub in clause2_list)
      if con == 'or':
        final_tuple = set(clause1_tuple) | set(clause2_tuple)
      elif con == 'and':
        final_tuple = set(clause1_tuple) & set(clause2_tuple)
    return final_tuple,len(final_tuple)






In [17]:
asso_rule.template3("1and2", "HEAD", "ANY", ['G10_Down'], "BODY", 2)

(set(), 0)

In [18]:
asso_rule.template3("2and2", "HEAD", 1, "BODY", 2)

({('G59_Up', 'G72_Up'), ('G59_Up', 'G82_Down')}, 2)

In [19]:
asso_rule.template3("1and2", "HEAD", 1, ['G82_Down','G96_Down','G13_Down', 'G59_Up'], "BODY", 1)#, ['G82_Down','G96_Down','G13_Down', 'G59_Up'])

({('G59_Up', 'G72_Up'), ('G59_Up', 'G82_Down'), ('G82_Down',), ('G96_Down',)},
 4)

In [20]:
asso_rule.template2("RULE", 5)

([], 0)

In [21]:
asso_rule.template1("RULE",4,['G82_Down','G96_Down','G13_Down', 'G59_Up'])

([], 0)

In [22]:
rule_list

[['G96_Down', '->', 'G59_Up', 'G72_Up'],
 ['G72_Up', 'G96_Down', '->', 'G59_Up'],
 ['G59_Up', 'G96_Down', '->', 'G72_Up'],
 ['G59_Up', 'G72_Up', '->', 'G96_Down'],
 ['G82_Down', '->', 'G59_Up', 'G72_Up'],
 ['G72_Up', '->', 'G59_Up', 'G82_Down'],
 ['G72_Up', 'G82_Down', '->', 'G59_Up'],
 ['G59_Up', 'G82_Down', '->', 'G72_Up'],
 ['G59_Up', 'G72_Up', '->', 'G82_Down']]

In [26]:
asso_rule.template1("RULE", "ANY", ['G59_Up'])

([['G96_Down', '->', 'G59_Up', 'G72_Up'],
  ['G72_Up', 'G96_Down', '->', 'G59_Up'],
  ['G59_Up', 'G96_Down', '->', 'G72_Up'],
  ['G59_Up', 'G72_Up', '->', 'G96_Down'],
  ['G82_Down', '->', 'G59_Up', 'G72_Up'],
  ['G72_Up', '->', 'G59_Up', 'G82_Down'],
  ['G72_Up', 'G82_Down', '->', 'G59_Up'],
  ['G59_Up', 'G82_Down', '->', 'G72_Up'],
  ['G59_Up', 'G72_Up', '->', 'G82_Down']],
 9)

In [27]:
asso_rule.template1("BODY", 1, ['G59_Up', 'G10_Down'])

([['G59_Up', 'G72_Up'],
  ['G59_Up'],
  ['G59_Up', 'G72_Up'],
  ['G59_Up', 'G82_Down'],
  ['G59_Up']],
 5)

In [28]:
asso_rule.template1("HEAD", "ANY", ['G59_Up'])

([['G59_Up', 'G96_Down'],
  ['G59_Up', 'G72_Up'],
  ['G59_Up', 'G82_Down'],
  ['G59_Up', 'G72_Up']],
 4)